In [2]:
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [3]:
Username = 'Username'
Password = 'Password'

In [4]:
driver = webdriver.Chrome() 

driver.get("https://accounts.spotify.com/en-GB/login?continue=https%3A%2F%2Fcharts.spotify.com/login")

time.sleep(1)
btnUser = driver.find_element(By.XPATH,'//*[@id="login-username"]')
btnUser.send_keys(Username)
btnPass = driver.find_element(By.XPATH,'//*[@id="login-password"]')
btnPass.send_keys(Password + Keys.ENTER)
time.sleep(5)
driver.get("https://charts.spotify.com/charts/view/regional-th-weekly/latest")
time.sleep(10) 

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

driver.quit()

In [5]:
soup_table = soup.find('table')

In [6]:
soup_name = soup_table.find_all('span',class_='styled__StyledTruncatedTitle-sc-135veyd-22 kKOJRc')
soup_artist = soup_table.find_all('div',class_='styled__StyledArtistsTruncatedDiv-sc-135veyd-28 ibVmha')
soup_rank = soup_table.find_all('span',class_='Type__TypeElement-goli3j-0 hgLxdb')
soup_href = soup_table.find_all('div',class_="styled__Wrapper-sc-135veyd-14 gPJpnT")
soup_rankweek = soup_table.find_all('td',class_='TableCell__TableCellElement-sc-1nn7cfv-0 dLdEGj styled__RightTableCell-sc-135veyd-4 kGfYTK')
soup_streams = soup_table.find_all('td',class_='TableCell__TableCellElement-sc-1nn7cfv-0 kJgiFu styled__RightTableCell-sc-135veyd-4 kGfYTK')

In [7]:
name_list = [item.text.strip() for item in soup_name]
artist_list = [item.text.strip() for item in soup_artist]
rank_list = [item.text.strip() for item in soup_rank]
href_list = [item.find('a')['href'].strip() for item in soup_href]
peak_list = [item.text.strip() for item in soup_rankweek[0::3]]
previous_list = [item.text.strip() for item in soup_rankweek[1::3]]
weeks_on_chart_list = [item.text.strip() for item in soup_rankweek[2::3]]
streams_list = [item.text.strip() for item in soup_streams]

In [8]:
import pandas as pd
top_song_200 = pd.DataFrame({
    'Rank':rank_list ,'Name Song': name_list, 'Artist': artist_list,
    'Peak Rank':peak_list, 'Previous Rank':previous_list, 
    'Weeks On Chart':weeks_on_chart_list, 'Streaming': streams_list, 'Uri':href_list
                            })
top_song_200

,Rank,Name Song,Artist,Peak Rank,Previous Rank,Weeks On Chart,Streaming,Uri
0,1,Like Crazy,Jimin,1,2,56,"2,116,360",https://open.spotify.com/track/3Ua0m0YmEjrMi9X...
1,2,Standing Next to You,Jung Kook,1,3,7,"2,036,630",https://open.spotify.com/track/2KslE17cAJNHTsI...
2,3,FRI(END)S,V,1,1,5,"1,862,164",https://open.spotify.com/track/0PBTlQ84CBJF0tB...
3,4,Love Me Again,V,1,5,36,"1,632,169",https://open.spotify.com/track/2N0SPREDYqILVEF...
4,5,Nightwalker,TEN,2,4,10,"1,280,779",https://open.spotify.com/track/4RiudH8RehvLLrk...
...,...,...,...,...,...,...,...,...
195,196,เทพลีลา,Silly Fools,196,—,1,"144,750",https://open.spotify.com/track/224KJQt1pd72n03...
196,197,เข้าใจว่าไม่เข้าใจ (INVERTED),Lula,148,161,4,"143,973",https://open.spotify.com/track/2phhqdmCi0p8XFV...
197,198,17,Dept,3,184,2,"143,095",https://open.spotify.com/track/3LejSFG20mSl4Iz...
198,199,Lie With You,TEN,51,—,1,"142,630",https://open.spotify.com/track/39uhLliuH6Otvhg...


# lyrics

In [37]:
import re

def clean_text(text):
    # Remove parentheses
    cleaned_text = re.sub(r'\([^()]*\)', ' ', text)
    
    cleaned_text = re.sub(r'เข้าระบบด้วย Facebook ', ' ', cleaned_text)
    
    cleaned_text = re.sub(r'เข้าระบบด้วย Facebook', '', cleaned_text)
    
    cleaned_text = cleaned_text.replace("\\", "")
    #cleaned_text = cleaned_text.replace(',', ' ')

    # Remove single quotes
    #cleaned_text = cleaned_text.replace("'", " ")
    
    # Remove double quotes
    cleaned_text = cleaned_text.replace('"', " ")
    
    # Remove excess whitespace
    cleaned_text = ' '.join(cleaned_text.split())
    
    return cleaned_text


In [10]:
def clean_only_text(text):
    # Remove parentheses
    cleaned_text = re.sub(r'[()]', '', text)
    
    return cleaned_text

In [11]:
def clean_genius_text(text):
    # Remove parentheses
    cleaned_text = re.sub(r'[()]', '', text)
    cleaned_text = re.sub(r' ', '-', cleaned_text)
    
    return cleaned_text

In [40]:
lyrics = []

In [41]:
#lyrics = []
# Importing necessary libraries
from selenium.common.exceptions import NoSuchElementException

# Setting up the webdriver
driver = webdriver.Chrome() 

# Recording the start time for performance tracking
start_time = time.time()

# Looping through a range of songs 
for i in range(len(top_song_200)): 
    print(f'Song {i+1}')
    song_name = top_song_200['Name Song'][i]
    artist_name = top_song_200['Artist'][i]

    # Search for lyrics using song name and artist
    driver.get(f"https://www.google.co.th/search?q=lyrics-{song_name}-{artist_name}")
    time.sleep(1) 
    print(f'Searching for lyrics of {song_name} by {artist_name}')

    # Retrieving HTML content of the page
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    lyric = ''
    for item in soup.find_all('div', class_="ujudUb"):
        for txt in item.find_all('span'):
            lyric += " " + txt.text.strip()
            
    # If lyrics are not found, try another search query
    if not lyric:
        driver.get(f"https://www.google.co.th/search?q=เนื้อเพลง+{clean_only_text(song_name)}")
        time.sleep(1) 
        print(f'Searching for lyrics of {song_name} in another attempt')

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        for item in soup.find_all('div', class_="ujudUb"):
            for txt in item.find_all('span'):
                lyric += " " + txt.text.strip()
                
    # If still not found, try Kapook search
    if not lyric:
        driver.get(f"https://www.google.co.th/search?q=เนื้อเพลง+{clean_only_text(song_name)}+kapook")
        time.sleep(1)
        try:
            btnkapook = driver.find_element(By.XPATH,'//*[@id="rso"]/div[1]/div/div/div[1]/div/div/span/a/div/div/div/div[1]/span').click()
        except:
            try:
                btnkapook = driver.find_element(By.XPATH,'//*[@id="rso"]/div[2]/div/div/div[1]/div/div/span/a/div/div/div/div[1]/span').click()
            except:
                try:
                    btnkapook = driver.find_element(By.XPATH,'//*[@id="rso"]/div[1]/div/div/div/div[1]/div/div/span/a/div/div/div/div[1]/span').click()
                except NoSuchElementException:
                    pass  # If none of the XPaths work, move on without clicking

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        for item in soup.find_all('p'):
            for txt in item:
                lyric += " " + txt.text.strip()
        
    # If lyrics are still not found, log the failure
    if not lyric:
        print(f'Failed to scrape lyrics for {song_name}')
    else:
        print(f'Successfully scraped lyrics for {song_name}')
        
    lyrics.append(lyric)

# Calculate total time taken and print
elapsed_time = time.time() - start_time
print(f"Total time taken: {elapsed_time} seconds")

# Quit the webdriver
driver.quit()

Song 1
Searching for lyrics of Like Crazy by Jimin
Successfully scraped lyrics for Like Crazy
Song 2
Searching for lyrics of Standing Next to You by Jung Kook
Successfully scraped lyrics for Standing Next to You
Song 3
Searching for lyrics of FRI(END)S by V
Successfully scraped lyrics for FRI(END)S
Song 4
Searching for lyrics of Love Me Again by V
Successfully scraped lyrics for Love Me Again
Song 5
Searching for lyrics of Nightwalker by TEN
Successfully scraped lyrics for Nightwalker
Song 6
Searching for lyrics of MONEY by LISA
Successfully scraped lyrics for MONEY
Song 7
Searching for lyrics of LALISA by LISA
Successfully scraped lyrics for LALISA
Song 8
Searching for lyrics of Seven (feat. Latto) (Explicit Ver.) by Jung Kook, Latto
Searching for lyrics of Seven (feat. Latto) (Explicit Ver.) in another attempt
Successfully scraped lyrics for Seven (feat. Latto) (Explicit Ver.)
Song 9
Searching for lyrics of 3D (feat. Jack Harlow) by Jung Kook, Jack Harlow
Successfully scraped lyrics 

Successfully scraped lyrics for พิจารณา (Consider)
Song 61
Searching for lyrics of สวยขยี้ใจ by บุ๊ค ศุภกาญจน์, ทิดแอม, มอส คำหมากบิน
Searching for lyrics of สวยขยี้ใจ in another attempt
Successfully scraped lyrics for สวยขยี้ใจ
Song 62
Searching for lyrics of ที่สุดเลย by RIFLE, Saran
Searching for lyrics of ที่สุดเลย in another attempt
Successfully scraped lyrics for ที่สุดเลย
Song 63
Searching for lyrics of บรรยากาศ by Only Monday
Searching for lyrics of บรรยากาศ in another attempt
Successfully scraped lyrics for บรรยากาศ
Song 64
Searching for lyrics of กลิ่นดอกไม้ by Newery
Searching for lyrics of กลิ่นดอกไม้ in another attempt
Successfully scraped lyrics for กลิ่นดอกไม้
Song 65
Searching for lyrics of กลับมาเพื่อบอกลา (JUST TO LET ME KNOW) by Paper Planes
Searching for lyrics of กลับมาเพื่อบอกลา (JUST TO LET ME KNOW) in another attempt
Successfully scraped lyrics for กลับมาเพื่อบอกลา (JUST TO LET ME KNOW)
Song 66
Searching for lyrics of พี่คงไม่ชอบผมหรอก by PONCHET, VARINZ
Success

Searching for lyrics of แฟนใหม่หน้าคุ้น by Maiyarap, MILLI
Searching for lyrics of แฟนใหม่หน้าคุ้น in another attempt
Successfully scraped lyrics for แฟนใหม่หน้าคุ้น
Song 118
Searching for lyrics of LIKE THAT by BABYMONSTER
Successfully scraped lyrics for LIKE THAT
Song 119
Searching for lyrics of สลักจิต by Pop Pongkool, Da Endorphine
Searching for lyrics of สลักจิต in another attempt
Successfully scraped lyrics for สลักจิต
Song 120
Searching for lyrics of ฉบับปรับปรุง by ZENTYARB, PUNYARB, LAZYLOXY
Searching for lyrics of ฉบับปรับปรุง in another attempt
Successfully scraped lyrics for ฉบับปรับปรุง
Song 121
Searching for lyrics of งานเต้นรำ by Musketeers
Searching for lyrics of งานเต้นรำ in another attempt
Successfully scraped lyrics for งานเต้นรำ
Song 122
Searching for lyrics of ไม่ยินดี by SAMBLACK
Successfully scraped lyrics for ไม่ยินดี
Song 123
Searching for lyrics of เส้นบางๆ by Indigo
Searching for lyrics of เส้นบางๆ in another attempt
Successfully scraped lyrics for เส้นบางๆ
S

Searching for lyrics of หมายความว่าอะไร by MEAN Band
Searching for lyrics of หมายความว่าอะไร in another attempt
Successfully scraped lyrics for หมายความว่าอะไร
Song 179
Searching for lyrics of Fall in Love by ARMOR
Searching for lyrics of Fall in Love in another attempt
Successfully scraped lyrics for Fall in Love
Song 180
Searching for lyrics of ข้างกัน (City) by Three Man Down, feat. ออม TELExTELEXs
Successfully scraped lyrics for ข้างกัน (City)
Song 181
Searching for lyrics of เมะ by THE TOYS
Searching for lyrics of เมะ in another attempt
Successfully scraped lyrics for เมะ
Song 182
Searching for lyrics of ก่อนตาย by Big Ass
Searching for lyrics of ก่อนตาย in another attempt
Successfully scraped lyrics for ก่อนตาย
Song 183
Searching for lyrics of HEY HEY (feat. อาภาพร นครสวรรค์) by MILLI, อาภาพร นครสวรรค์
Searching for lyrics of HEY HEY (feat. อาภาพร นครสวรรค์) in another attempt
Successfully scraped lyrics for HEY HEY (feat. อาภาพร นครสวรรค์)
Song 184
Searching for lyrics of Espres

In [42]:
lyrics

[' She\'s saying, "Baby, 생각하지 마" There\'s not a bad thing here tonight Baby, 떠나도 좋아 있어 줘 오늘까지만 Watch me go, 날 적셔 밤새도록, away 아침도 취해서 오지 않게 (않게) 시끄러운 음악 속에 희미해진 나 드라마같은 뻔한 story 익숙해져 가 네가 알던 나를 찾기엔 멀리 온 걸까? Yeah, I know, you know, I know (ooh) I\'d rather be lost in the lights, lost in the lights I\'m out of my mind, 이 밤의 끝을 잡아줘 매일 밤, you spin me up high 너를 품은 달, let me have a taste Give me a good ride (Oh, I\'m fallin\', I\'m fallin\', I\'m fallin\') Oh, it\'s gon\' be a good night (Oh, I\'m fallin\') forever, you and I Hmm-hmm, yeah, yeah Hmm-hmm, yeah, yeah Hmm-hmm, yeah, yeah Hmm-hmm (forever, you and I) 거울 속에 비친 나 하염없이 미쳐가 I\'m feelin\' so alive, wasting time I\'d rather be lost in the lights, lost in the lights I\'m out of my mind, 이 밤의 끝을 잡아줘 매일 밤, you spin me up high 너를 품은 달, let me have a taste Give me a good ride (Oh, I\'m fallin\', I\'m fallin\', I\'m fallin\') Oh, it\'s gon\' be a good night (Oh, I\'m fallin\') forever, you and I This will break me, this is gonna break me (br

In [43]:
clean_lyrics = [ clean_text(i) for i in lyrics]
print(clean_lyrics)

["She's saying, Baby, 생각하지 마 There's not a bad thing here tonight Baby, 떠나도 좋아 있어 줘 오늘까지만 Watch me go, 날 적셔 밤새도록, away 아침도 취해서 오지 않게 시끄러운 음악 속에 희미해진 나 드라마같은 뻔한 story 익숙해져 가 네가 알던 나를 찾기엔 멀리 온 걸까? Yeah, I know, you know, I know I'd rather be lost in the lights, lost in the lights I'm out of my mind, 이 밤의 끝을 잡아줘 매일 밤, you spin me up high 너를 품은 달, let me have a taste Give me a good ride Oh, it's gon' be a good night forever, you and I Hmm-hmm, yeah, yeah Hmm-hmm, yeah, yeah Hmm-hmm, yeah, yeah Hmm-hmm 거울 속에 비친 나 하염없이 미쳐가 I'm feelin' so alive, wasting time I'd rather be lost in the lights, lost in the lights I'm out of my mind, 이 밤의 끝을 잡아줘 매일 밤, you spin me up high 너를 품은 달, let me have a taste Give me a good ride Oh, it's gon' be a good night forever, you and I This will break me, this is gonna break me No, don't you wake me I wanna stay in this dream Don't save me, don't you try to save me I need a way we I need a way we can dream on", "Standing next to you Play me slow I push up on this f

In [44]:
len(clean_lyrics)

200

## TEST save csv

In [45]:
test_200 = top_song_200

In [46]:
test_200['Lyrics'] = clean_lyrics

In [47]:
(test_200['Lyrics'] == '').sum()

3

In [48]:
test_200.to_csv('/Users/art/Desktop/เรียน/ปี 3 เทอม 2/CPE393 TEXT/Project/top_song_200_1.csv',index=False)

In [49]:
test_200[test_200['Lyrics'] == '']

,Rank,Name Song,Artist,Peak Rank,Previous Rank,Weeks On Chart,Streaming,Uri,Lyrics
40,41,ที่รักใจเย็น,"YENTED, BlackHeart, Williams",41,43,7,"389,504",https://open.spotify.com/track/6rgF7lJWhf0jR5Z...,
73,74,รสหวาน,FREEHAND,28,65,75,"287,455",https://open.spotify.com/track/2Ab1HQjbMcqHY3T...,
190,191,บิดเบือน,มนัสวีร์,179,179,2,"146,525",https://open.spotify.com/track/0xBStDSuDLyjqwa...,


In [66]:
test_200['Lyrics'][test_200['Name Song'] == 'ที่รักใจเย็น'] = 'Oh ที่รักช่วยใจเย็น ๆ ก่อน อย่าใจร้อนด่วนตัดสิน ช่วยฟังผมอธิบาย สิ่งที่คุณเห็นมันไม่ได้มีอะไร แฟนคลับสาว ๆ โทษทีที่เยอะเกินไป จนอาจจะทำให้คุณเข้าใจตัวผมผิดไป อยากจะขออภัยที่ทำให้ต้องมาทุกข์ใจอะไรไม่รู้ แค่เธอ มีแค่ มีแค่เธอเลย แค่เธอ มีแค่ มีแค่ เธอเลย Baby มีแค่เธอ Baby ผมหน่ะมีแค่เธอเลย I Got too many girl แต่ว่าไม่ได้คิดอะไรที่เกินเลย แค่ในบางเวลามีคนเข้ามา ผมคงไม่เมินเฉย I’m ma superstar แต่กลับบ้านมาก็มีแค่เธอเลย Cuz You are my type ถ้ามีเธอผมไม่ต้องการอะไร อยากให้รู้ว่าในใจมีแค่เธอ Baby girl เธอหน่ะคือคนที่ใช่ บอกกับสาว ๆ ไม่ต้องมาทัก ผมไม่ใช่ casnova boy เพราะเธอคือคนเดียว ที่ผมจะรัก ผมจะไม่ทำให้เธอต้องคอย และผมไม่เคยจะหวั่นไหว ต่อให้คนมารายล้อม ไอ้เรามันก็เท่ ซะด้วย อาจจะไปทำให้คนมากมายร้อง แต่ว่าเธอคือคนที่ผมจะให้ทั้งตัวและใจพร้อม Baby girl ไม่เอาไม่ร้อง Oh ที่รักช่วยใจเย็น ๆ ก่อน อย่าใจร้อนด่วนตัดสิน ช่วยฟังผมอธิบาย สิ่งที่คุณเห็นมันไม่ได้มีอะไร แฟนคลับสาว ๆ โทษทีที่เยอะเกินไป จนอาจจะทำให้คุณเข้าใจตัวผมผิดไป อยากจะขออภัยที่ทำให้ต้องมาทุกข์ใจอะไรไม่รู้ แค่เธอ มีแค่ มีแค่เธอเลย แค่เธอ มีแค่ มีแค่ เธอเลย คำพูดนินทาคนอื่น baby เธอหน่ะพอได้หรือยัง Baby ฉันมีแค่เธอจริง ๆ เธอไม่ต้องคอยระวัง ฉันรู้ว่าเธอนั้นนั่งน้อยใจ หรือว่ารักฉันมันน้อยไป เเค่อยากหวังให้เธอเข้าใจ เพราะวันนี้ฉันขอ ขอเถอะ baby เธอไม่ต้องระเเวง I hope you okay I can see you stand That all be fine Girl I hope you understand But you don’t understand Hey ที่รัก ไม่อยากให้คุณเข้าใจผมผิดเลย ขอโทษที่งานของผมมันทำให้คุณอึดอัดนะ ถ้ารู้สึกดีขึ้นแล้ว โทรกลับมาหาผมนะ ผมคิดถึงคุณมาก ๆ เลย Oh ที่รักช่วยใจเย็น ๆ ก่อน อย่าใจร้อนด่วนตัดสิน ช่วยฟังผมอธิบาย สิ่งที่คุณเห็นมันไม่ได้มีอะไร แฟนคลับสาว ๆ โทษทีที่เยอะเกินไป จนอาจจะทำให้คุณเข้าใจตัวผมผิดไป อยากจะขออภัยที่ทำให้ต้องมาทุกข์ใจอะไรไม่รู้ แค่เธอ มีแค่ มีแค่เธอเลย แค่เธอ มีแค่ มีแค่ เธอเลย'

/var/folders/q2/wft17t114qn0gp5mydmpqy900000gn/T/ipykernel_15143/2785711285.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_200['Lyrics'][test_200['Name Song'] == 'ที่รักใจเย็น'] = 'Oh ที่รักช่วยใจเย็น ๆ ก่อน อย่าใจร้อนด่วนตัดสิน ช่

In [68]:
test_200['Lyrics'][test_200['Name Song'] == 'รสหวาน'] = 'กลิ่นดอกไม้บอกฤดูกาล ลมหน้าหนาวพาฉันย้อนไป ที่เธอนั้น โอบกอดฉัน ไว้ข้างกาย แววตาของเธอช่างงดงาม จับมือฉันอยู่แค่มีเรา เรื่องวันนั้น เวลานั้น ที่ฉันมีเธอ คนที่เคยกอดฉันผ่านเรื่องราวเลวร้าย ให้มันคอยบอกฉันว่า ชีวิตที่เคยมีเธอมีค่า ให้มันยังคงสวยงามอยู่เช่นเดิม แม้มันจะเหลือแค่เพียงร่องรอยจาง ๆ รอยจูบเธอยังประทับลงในหัวใจ ให้ฉันยังได้ชิมรสหวานที่ทรมาน ดั่งแผลเป็นที่ช่วยเตือนว่าครั้งหนึ่งฉันเคยพบเธอและร่ำลา ต่อให้เริ่มใหม่และจบด้วยน้ำตาอีกครั้ง หากต้องย้อนไปลบทุกสิ่ง อาจไม่เข้าใจความสวยงาม ในบาดแผล และความรัก ไม่เข้าใจ ให้มันเป็นรอยแผลเก่า เก็บเรื่องราวเธอไว้ ให้มันคอยบอกฉันว่า ชีวิตที่เคยมีเธอมีค่า ให้มันยังคงสวยงามอยู่เช่นเดิม แม้มันจะเหลือแค่เพียงร่องรอยจาง ๆ รอยจูบเธอยังประทับลงในหัวใจ ให้ฉันยังได้ชิมรสหวานที่ทรมาน ดั่งแผลเป็นที่ช่วยเตือนว่าครั้งหนึ่งฉันเคยพบเธอและร่ำลา ต่อให้เริ่มใหม่และจบด้วยน้ำตาอีกครั้ง'

/var/folders/q2/wft17t114qn0gp5mydmpqy900000gn/T/ipykernel_15143/1806435770.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_200['Lyrics'][test_200['Name Song'] == 'รสหวาน'] = 'กลิ่นดอกไม้บอกฤดูกาล ลมหน้าหนาวพาฉันย้อนไป ที่เธอนั้น โอ

In [69]:
test_200['Lyrics'][test_200['Name Song'] == 'บิดเบือน'] = 'เธอไม่ต้องบอกอะไรทั้งนั้น กับเรื่องราวที่กำลังเกิด ที่เธอกับฉันร่วมกันเผชิญ และคงไม่ต้องบอกอะไรใช่ไหม กับสิ่งที่เธอได้ยิน กับสิ่งที่เธอได้ฟัง ไม่มีคำไหนจะพูดให้เธอเข้าใจ ไม่มีสิ่งไหนแสดงออกไป และไม่อยากบังคับให้เธอต้องเชื่อฉัน ได้โปรดอย่ามาทำร้ายใจกันอีกเลย อย่าปล่อยให้ความคิดของเธอถูกบิดเบือน ได้โปรดเถิดเธอลองใช้ใจเธอดู ก่อนจะปล่อยให้ใครตัดสินว่าฉันเป็นคนไม่ดี อย่างที่ใครเข้าใจ ไม่มีคำไหนจะพูดให้เธอเข้าใจ ไม่มีสิ่งไหนแสดงออกไป และไม่อยากบังคับให้เธอต้องเชื่อฉัน ได้โปรดอย่ามาทำร้ายใจกันอีกเลย อย่าปล่อยให้ความคิดของเธอถูกบิดเบือน ได้โปรดเถิดเธอลองใช้ใจเธอดู ก่อนจะปล่อยให้ใครตัดสินว่าฉันเป็นคนไม่ดี อย่างที่ใครเข้าใจ แต่ก็ไม่เป็นไรเพียงอยากให้เธอคิดทบทวนอีกครั้ง ได้โปรดอย่ามาทำร้ายใจกันอีกเลย อย่าปล่อยให้ความคิดของเธอถูกบิดเบือน ได้โปรดเถิดเธอลองใช้ใจเธอดู ก่อนจะปล่อยให้ใครตัดสินว่าฉันเป็นคนไม่ดี อย่างที่ใครเข้าใจ'

/var/folders/q2/wft17t114qn0gp5mydmpqy900000gn/T/ipykernel_15143/989959765.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_200['Lyrics'][test_200['Name Song'] == 'บิดเบือน'] = 'เธอไม่ต้องบอกอะไรทั้งนั้น กับเรื่องราวที่กำลังเกิด ที่เ

In [79]:
test_200[test_200['Lyrics'] == '']

,Rank,Name Song,Artist,Peak Rank,Previous Rank,Weeks On Chart,Streaming,Uri,Lyrics


In [71]:
test_200.to_csv('/Users/art/Desktop/เรียน/ปี 3 เทอม 2/CPE393 TEXT/Project/top_song_200_1.csv',index=False)

In [75]:
import re

line = [u'x', u'y', u'z', u'쭌', u'a']

if any([re.search(u'[\u3131-\ucb4c]', x) for x in line[3:]]):
    print("found character")

found character


In [88]:
import re
test_200['Name Song'][test_200['Lyrics'].str.contains('[\u3131-\ucb4c가-힣]+', flags=re.UNICODE)]


0                            Like Crazy
3                         Love Me Again
5                                 MONEY
6                                LALISA
9                              Magnetic
11     NEURON (with Gaeko & YOON MIRAE)
14                                  TAP
19                              Haegeum
21                               SHEESH
22                           Pink Venom
44                                Smart
48                              Deja Vu
58          Michi Teyu Ku (Overflowing)
68                                Abyss
71                     Set Me Free Pt.2
79                        The Astronaut
87                     Closer Than This
89                          Midas Touch
91                           Rainy Days
96                         Slow Dancing
100                          Super Tuna
108          Wild Flower (with youjeen)
113                                EASY
127      อาหมวยหาย (阿妹走 / Chinese Girl)
158                           Super Shy


In [89]:
import re
test_200['Name Song'][test_200['Lyrics'].str.contains('อัลบั้ม', flags=re.UNICODE)]


63                 กลิ่นดอกไม้
74     เรื่องราวความรักในตำนาน
84           ซูลูปาก้า ตาปาเฮ้
111               จะมอบความรัก
170                   ลองฟังแล
Name: Name Song, dtype: object